In [30]:
include("src/NursesScheduling.jl")
using .NurseSchedules
using Logging

logger = ConsoleLogger(stderr, Logging.Debug)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(0x00000030) open, 0 bytes waiting))), Debug, Logging.default_metafmt, true, 0, Dict{Any,Int64}())

In [31]:
schedule = Schedule("./schedules/schedule_2016_august.json")

Schedule(Dict{String,Any}("shifts" => Dict{String,Any}("nurse_9" => Any["U", "U", "U", "U", "U", "W", "DN", "W", "DN", "W"  …  "DN", "W", "DN", "N", "W", "DN", "U", "U", "U", "U"],"nurse_2" => Any["W", "D", "R", "N", "W", "D", "W", "N", "W", "N"  …  "W", "DN", "PN", "W", "DN", "W", "W", "D", "R", "N"],"nurse_3" => Any["U", "U", "U", "U", "W", "DN", "W", "DN", "W", "DN"  …  "W", "W", "W", "D", "N", "W", "U", "U", "U", "U"],"nurse_6" => Any["U", "U", "U", "U", "U", "U", "U", "U", "U", "U"  …  "W", "D", "DN", "N", "W", "DN", "U", "U", "U", "U"],"nurse_8" => Any["D", "N", "N", "W", "D", "W", "W", "D", "N", "W"  …  "R", "W", "W", "D", "W", "W", "D", "N", "N", "W"],"nurse_4" => Any["W", "W", "D", "D", "N", "W", "D", "N", "W", "D"  …  "U", "U", "U", "U", "W", "W", "W", "W", "D", "D"],"nurse_5" => Any["W", "W", "W", "W", "N", "W", "W", "W", "W", "W"  …  "DN", "N", "R", "W", "PN", "W", "W", "W", "W", "W"],"nurse_7" => Any["N", "W", "N", "W", "U", "U", "U", "U", "U", "U"  …  "W", "N", "W", "N", 

In [32]:
workers, shifts = get_shifts(schedule)
month_info = get_month_info(schedule)
workers_info = get_workers_info(schedule)

Dict{String,Any} with 1 entry:
  "time" => Dict{String,Any}("nurse_9"=>"FULL","nurse_2"=>"FULL","nurse_3"=>"FU…

In [33]:
with_logger(logger) do
    get_max_nbhd_size(schedule)
end

┌ Debug: Neighbors number from addition: 768
└ @ Main.NurseSchedules.Neighborhood ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:11
┌ Debug: Neighbors number from deletion: 118
└ @ Main.NurseSchedules.Neighborhood ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:13
┌ Debug: Neighbors number from swap: 4453
└ @ Main.NurseSchedules.Neighborhood ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:26


5339

In [34]:
neighborhood = get_nbhd(shifts)
length(neighborhood)

5339

In [35]:
with_logger(logger) do
    global schedule_score = score(workers, shifts, month_info, workers_info)
end

┌ Debug: Lacking nurses on day '1'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '1' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '2'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '1' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '3'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '2' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '4'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '1' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '5'.
│ Expected '4', got '1' in the day.
│ Expected '3', got '2' at night.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/s

┌ Debug: Worker 'nurse_9' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:104
┌ Debug: Worker 'nurse_9' does not have a long break in week: '2'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:104
┌ Debug: Worker 'nurse_9' does not have a long break in week: '3'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:104
┌ Debug: Worker 'nurse_9' does not have a long break in week: '4'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:104
┌ Debug: Worker 'nurse_9' does not have a long break in week: '5'
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:104
┌ Debug: Worker 'nurse_2' has a disallowed shift sequence in day '27': DN -> PN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ D

3050

In [36]:
ITERATION_NUM = 1000
best_shifts = (shifts, schedule_score)

(["U" "U" … "U" "U"; "W" "D" … "R" "N"; … ; "N" "W" … "N" "W"; "U" "U" … "U" "U"], 3050)

In [37]:
iter_best = best_shifts
no_improvement_iters = 0

for i in 1:ITERATION_NUM
    neighborhood = get_nbhd(iter_best[1])
    nghd_scores = map(s -> score(workers, s, month_info, workers_info), neighborhood)
    iter_best_idx = findfirst(nghd_scores .== minimum(nghd_scores))
    iter_best = (neighborhood[iter_best_idx], nghd_scores[iter_best_idx])
    
    no_improvement_iters += 1
    
    if best_shifts[2] > iter_best[2]
        println("Improved from $(best_shifts[2]) to $(iter_best[2]) in iteration '$(i)'")
        global no_improvement_iters = 0
        global best_shifts = iter_best
    end
    if no_improvement_iters > 20
        println("Local minimum! Stopping in iteration '$(i)'")
        break
    end
end

Improved from 3050 to 3000 in iteration '1'
Improved from 3000 to 2950 in iteration '2'
Improved from 2950 to 2900 in iteration '3'
Improved from 2900 to 2850 in iteration '4'
Improved from 2850 to 2801 in iteration '5'
Improved from 2801 to 2752 in iteration '6'
Improved from 2752 to 2712 in iteration '7'
Improved from 2712 to 2672 in iteration '8'
Improved from 2672 to 2632 in iteration '9'
Improved from 2632 to 2592 in iteration '10'
Improved from 2592 to 2552 in iteration '11'
Improved from 2552 to 2512 in iteration '12'
Improved from 2512 to 2472 in iteration '13'
Improved from 2472 to 2432 in iteration '14'
Improved from 2432 to 2392 in iteration '15'
Improved from 2392 to 2352 in iteration '16'
Improved from 2352 to 2312 in iteration '17'
Improved from 2312 to 2272 in iteration '18'
Improved from 2272 to 2232 in iteration '19'
Improved from 2232 to 2192 in iteration '20'
Improved from 2192 to 2152 in iteration '21'
Improved from 2152 to 2112 in iteration '22'
Improved from 2112 

In [38]:
with_logger(logger) do
    score(workers, best_shifts[1], month_info, workers_info)
end

┌ Debug: Lacking nurses on day '3'.
│ Expected '4', got '3' in the day.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '5'.
│ Expected '4', got '3' in the day.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Lacking nurses on day '34'.
│ Expected '4', got '3' in the day.
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:61
┌ Debug: Total lack of nurses penalty: 60
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:65
┌ Debug: Worker 'nurse_9' has a disallowed shift sequence in day '6': N -> DN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ Debug: Worker 'nurse_9' has a disallowed shift sequence in day '7': DN -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ D

┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '2': DN -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '15': N -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '19': DN -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '22': DN -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '32': DN -> DN
└ @ Main.NurseSchedules.ScheduleScore ~/Documents/nurse/nurse_scheduling_problem/src/score.jl:86
┌ Debug: Worker 'nurse_4' has a disallowed shift sequence in day '33': DN -> D
└ @ Main.NurseSchedules.ScheduleScore ~/Do

659